In [1]:
import pytorch_lightning as pl
from omegaconf import OmegaConf, DictConfig
import nemo.collections.asr as nemo_asr
from nemo.core.config import hydra_runner
from nemo.utils import logging
from nemo.utils.exp_manager import exp_manager
import datetime
from pathlib import Path
import torch
from sbb_project import consts
import copy
from pytorch_lightning.loggers import WandbLogger  
import wandb
from ruamel.yaml import YAML
import functools
import copy

[NeMo W 2023-01-02 18:38:38 optimizers:67] Could not import distributed_fused_adam optimizer from Apex
[NeMo W 2023-01-02 18:38:40 nemo_logging:349] /home/user/.local/lib/python3.9/site-packages/torch/jit/annotations.py:309: UserWarning: TorchScript will treat type annotations of Tensor dtype-specific subtypes as if they are normal Tensors. dtype constraints are not enforced in compilation either.
      warnings.warn("TorchScript will treat type annotations of Tensor "
    


In [2]:
sweep_config = {
  "method": "random",   # Random search
  "metric": {           # We want to minimize `val_loss`
      "name": "val_wer",
      "goal": "minimize"
  },
  "parameters": {
        "lr": {
            # log uniform distribution between exp(min) and exp(max)
            "distribution": "log_uniform",
            "min": -9.21,   # exp(-9.21) = 1e-4
            "max": -2.30    # exp(-4.61) = 1e-2
        },
      "freq_masks": {
            "distribution": "int_uniform",
            "min": 0,  
            "max": 7     
        },
      "time_masks": {
            "distribution": "int_uniform",
            "min": 0,  
            "max": 15     
        },
        "freq_width": {
            "distribution": "uniform",
            "min": 25,  
            "max": 30     
        },
          "time_width": {
                "distribution": "uniform",
                "min": 0.01,  
                "max": 0.07     
            }      
    }
}

In [3]:
train_manifest = [
        str(consts.MANIFEST_DIR.joinpath(consts.MANIFEST_FILE.format("train", "none"))),
        str(consts.MANIFEST_DIR.joinpath(consts.MANIFEST_FILE.format("train", "neg10"))),
        str(consts.MANIFEST_DIR.joinpath(consts.MANIFEST_FILE.format("train", "0"))),
        str(consts.MANIFEST_DIR.joinpath(consts.MANIFEST_FILE.format("train", "10"))),
        str(consts.MANIFEST_DIR.joinpath(consts.MANIFEST_FILE.format("train", "20"))),
        str(consts.MANIFEST_DIR.joinpath(consts.MANIFEST_FILE.format("train", "30"))),
        str(consts.MANIFEST_DIR.joinpath(consts.MANIFEST_FILE.format("train", "40"))),
    ]
val_manifest = consts.MANIFEST_DIR.joinpath(consts.MANIFEST_FILE.format("val", "random"))
test_manifest = consts.MANIFEST_DIR.joinpath(consts.MANIFEST_FILE.format("test", "random"))

In [4]:
def sweep_iteration():
    
    # set up W&B logger
    wandb.init()    # required to have access to `wandb.config`
    wandb_logger = WandbLogger(log_model='all')  # log final model
        
    trainer = pl.Trainer(max_epochs=10, logger=wandb_logger, devices = [1], accelerator="gpu")
    
    # setup model - note how we refer to sweep parameters with wandb.config
    model = nemo_asr.models.ASRModel.from_pretrained(model_name='stt_de_conformer_transducer_large',
                                                    map_location=torch.device("cuda:1"))

    model.set_trainer(trainer)
    
    model.cfg.train_ds.is_tarred = False
    
    model.cfg.train_ds.manifest_filepath = ','.join(train_manifest)
    model.cfg.validation_ds.manifest_filepath = str(val_manifest)
    model.cfg.test_ds.manifest_filepath = str(test_manifest)
    
    model.cfg.train_ds.max_duration = 45
    model.cfg.train_ds.batch_size = 1
    model.cfg.validation_ds.batch_size = 1
    model.cfg.test_ds.batch_size = 1
    
    model.cfg.optim.lr = wandb.config.lr
    model.cfg.spec_augment.freq_width = wandb.config.freq_width
    model.cfg.spec_augment.freq_masks = wandb.config.freq_masks
    model.cfg.spec_augment.time_width = wandb.config.time_width
    model.cfg.spec_augment.time_masks = wandb.config.time_masks

    model.setup_training_data(model.cfg.train_ds)
    model.setup_validation_data(model.cfg.validation_ds)
    model.setup_test_data(model.cfg.test_ds)
    model.setup_optimization(model.cfg.optim)

    # train
    trainer.fit(model)


In [ ]:
sweep_id = wandb.sweep(sweep_config, project="ASR-Conformer-TD")
wandb.agent(sweep_id, function=sweep_iteration)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING Malformed sweep config detected! This may cause your sweep to behave in unexpected ways.
wandb: WARNING To avoid this, please fix the sweep config schema violations below:
wandb: WARNING   Violation 1. lr uses log_uniform, where min/max specify base-e exponents. Use log_uniform_values to specify limit values.


Create sweep with ID: x72jj9z5
Sweep URL: https://wandb.ai/unisg-ds-nlp/ASR-Conformer-TD/sweeps/x72jj9z5


wandb: Agent Starting Run: kq7zq1g7 with config:
wandb: 	freq_masks: 2
wandb: 	freq_width: 28.354047675750778
wandb: 	lr: 0.012118745624772182
wandb: 	time_masks: 4
wandb: 	time_width: 0.05949828281723428
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: marccgrau (unisg-ds-nlp). Use `wandb login --relogin` to force relogin


[NeMo W 2023-01-02 18:38:48 nemo_logging:349] /home/user/.local/lib/python3.9/site-packages/pytorch_lightning/loggers/wandb.py:396: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
      rank_zero_warn(
    
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


[NeMo I 2023-01-02 18:38:48 cloud:66] Downloading from: https://api.ngc.nvidia.com/v2/models/nvidia/nemo/stt_de_conformer_transducer_large/versions/1.5.0/files/stt_de_conformer_transducer_large.nemo to /home/user/.cache/torch/NeMo/NeMo_1.14.0/stt_de_conformer_transducer_large/183f232a57704e78fd9b211a3e021f02/stt_de_conformer_transducer_large.nemo
100% [......................................................................] 447627250 / 447627250[NeMo I 2023-01-02 18:39:18 common:912] Instantiating model from pre-trained checkpoint
[NeMo I 2023-01-02 18:39:23 mixins:170] Tokenizer SentencePieceTokenizer initialized with 1024 tokens


...

[NeMo I 2023-01-02 22:59:45 rnnt_wer_bpe:390] 
    
[NeMo I 2023-01-02 22:59:45 rnnt_wer_bpe:391] reference :rangierfahrt vom gleis mike zwei drei ins drei drei antworten
[NeMo I 2023-01-02 22:59:45 rnnt_wer_bpe:392] predicted :ermatte rangierfragt von gleis mike zwei drei autonts an sich
[NeMo I 2023-01-02 22:59:45 rnnt_wer_bpe:390] 
    
[NeMo I 2023-01-02 22:59:45 rnnt_wer_bpe:391] reference :rangierfahrt von whisky zwei fünf nach drei null antworten
[NeMo I 2023-01-02 22:59:45 rnnt_wer_bpe:392] predicted :rangierfahrt von whisky zwei fünf nach drei no antworten
[NeMo I 2023-01-02 22:59:46 rnnt_wer_bpe:390] 
    
[NeMo I 2023-01-02 22:59:46 rnnt_wer_bpe:391] reference :umstellmanöver von november neun nach fünf via gleis tango vier und romeo drei antworten
[NeMo I 2023-01-02 22:59:46 rnnt_wer_bpe:392] predicted :umstellmanöver von november neun noch fünf vier gleis tango vier hund romeo drei antworten
[NeMo I 2023-01-02 22:59:46 rnnt_wer_bpe:390] 
    
[NeMo I 2023-01-02 22:59:46 rn

Validation: 0it [00:00, ?it/s]

[NeMo I 2023-01-02 23:07:43 rnnt_wer_bpe:390] 
    
[NeMo I 2023-01-02 23:07:43 rnnt_wer_bpe:391] reference :rangierfahrt vom gleis mike zwei drei ins drei drei antworten
[NeMo I 2023-01-02 23:07:43 rnnt_wer_bpe:392] predicted :er rangierfond von gleis mike zwei drei auton
[NeMo I 2023-01-02 23:07:44 rnnt_wer_bpe:390] 
    
[NeMo I 2023-01-02 23:07:44 rnnt_wer_bpe:391] reference :rangierfahrt von whisky zwei fünf nach drei null antworten
[NeMo I 2023-01-02 23:07:44 rnnt_wer_bpe:392] predicted :rangierfahrt sonn whisky zwei fünf nach drei no antworten
[NeMo I 2023-01-02 23:07:44 rnnt_wer_bpe:390] 
    
[NeMo I 2023-01-02 23:07:44 rnnt_wer_bpe:391] reference :umstellmanöver von november neun nach fünf via gleis tango vier und romeo drei antworten
[NeMo I 2023-01-02 23:07:44 rnnt_wer_bpe:392] predicted :umstellmanöver von november neun noch fünf vier gleiso vier hund romeo drei antworten
[NeMo I 2023-01-02 23:07:44 rnnt_wer_bpe:390] 
    
[NeMo I 2023-01-02 23:07:44 rnnt_wer_bpe:391] refe

Validation: 0it [00:00, ?it/s]

[NeMo I 2023-01-02 23:15:41 rnnt_wer_bpe:390] 
    
[NeMo I 2023-01-02 23:15:41 rnnt_wer_bpe:391] reference :rangierfahrt vom gleis mike zwei drei ins drei drei antworten
[NeMo I 2023-01-02 23:15:41 rnnt_wer_bpe:392] predicted :irgend rangier vom gleis mike zwei drei ins dreiton
[NeMo I 2023-01-02 23:15:41 rnnt_wer_bpe:390] 
    
[NeMo I 2023-01-02 23:15:41 rnnt_wer_bpe:391] reference :rangierfahrt von whisky zwei fünf nach drei null antworten
[NeMo I 2023-01-02 23:15:41 rnnt_wer_bpe:392] predicted :rangierfahrt sonn whisky zwei fünf nach drei no antworten
[NeMo I 2023-01-02 23:15:42 rnnt_wer_bpe:390] 
    
[NeMo I 2023-01-02 23:15:42 rnnt_wer_bpe:391] reference :umstellmanöver von november neun nach fünf via gleis tango vier und romeo drei antworten
[NeMo I 2023-01-02 23:15:42 rnnt_wer_bpe:392] predicted :umstellmanöver von november neun nach fünf vier gleiso vier hund romeo drei antworten
[NeMo I 2023-01-02 23:15:42 rnnt_wer_bpe:390] 
    
[NeMo I 2023-01-02 23:15:42 rnnt_wer_bpe:391